## download the data 

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files 
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mayarhanafy","key":"b582afeab0138449dd11ed1563e88028"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d 'lakshmi25npathi/imdb-dataset-of-50k-movie-reviews'

 66% 17.0M/25.7M [00:00<00:00, 63.3MB/s]
100% 25.7M/25.7M [00:00<00:00, 86.5MB/s]


In [ ]:
! unzip -x "/content/imdb-dataset-of-50k-movie-reviews.zip"

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
import numpy as np
import pandas as pd
import pylab as plt
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import cohen_kappa_score

from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

from imblearn.over_sampling import SMOTE
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import io
import requests
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
url = 'IMDB Dataset.csv'
data = pd.read_csv(url)

In [ ]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
data = data.dropna().reset_index(drop=True)

In [ ]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
sentiments = data['sentiment'].value_counts()
print('Sentiments in entire dataset:\n Positive: {}\n Negative: {}'.format(sentiments[1], sentiments[0]))

Sentiments in entire dataset:
 Positive: 25000
 Negative: 25000


## word2vec

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import re
corups=[]
pos=[]
for i in range (len(data)):
  comment=data['review'][i]
  comment=comment.lower()
  comment=comment.replace('<br /><br />',' ')
  comment=re.sub('[^a-zA-Z]',' ',comment)

  l=[word for word in word_tokenize(comment) if word not in stopwords.words('english')]
  pos.append(nltk.pos_tag(l))
  comment=' '.join(l)
  corups.append(comment)

In [ ]:
corups[0]

'one reviewers mentioned watching oz episode hooked right exactly happened first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home many aryans muslims gangstas latinos christians italians irish scuffles death stares dodgy dealings shady agreements never far away would say main appeal show due fact goes shows dare forget pretty pictures painted mainstream audiences forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards sold nickel inmates kill order get away well mannered middle class inmates turned prison bitches due lack street skill

In [ ]:
tag=[]
words = []
for i in pos :
  t=[]
  w=[]
  for j in i:
    if j[1] not in ['VB','VBD','VBG','VBZ','VBP','VBN']:
        w.append(j[0])
        t.append(j[1])
  tag.append(t)
  words.append(w)

In [ ]:
words

[['one',
  'reviewers',
  'oz',
  'episode',
  'right',
  'exactly',
  'first',
  'thing',
  'oz',
  'brutality',
  'scenes',
  'violence',
  'right',
  'word',
  'trust',
  'show',
  'faint',
  'timid',
  'show',
  'punches',
  'regards',
  'drugs',
  'sex',
  'violence',
  'hardcore',
  'classic',
  'use',
  'word',
  'oz',
  'nickname',
  'oswald',
  'maximum',
  'security',
  'state',
  'penitentary',
  'focuses',
  'mainly',
  'city',
  'experimental',
  'section',
  'prison',
  'cells',
  'glass',
  'fronts',
  'inwards',
  'high',
  'agenda',
  'city',
  'home',
  'many',
  'aryans',
  'muslims',
  'latinos',
  'christians',
  'italians',
  'irish',
  'scuffles',
  'death',
  'stares',
  'dealings',
  'shady',
  'agreements',
  'never',
  'far',
  'away',
  'would',
  'main',
  'appeal',
  'show',
  'due',
  'fact',
  'shows',
  'pretty',
  'pictures',
  'mainstream',
  'audiences',
  'charm',
  'forget',
  'romance',
  'oz',
  'mess',
  'around',
  'first',
  'episode',
  'ever

In [ ]:
import gensim

In [ ]:
w = gensim.models.Word2Vec(words,size=200)
t= gensim.models.Word2Vec(tag,size=200)

In [ ]:
w.wv.similar_by_word('film')

[('movie', 0.6146670579910278),
 ('films', 0.5719402432441711),
 ('cinema', 0.5189324617385864),
 ('nevertheless', 0.5046309232711792),
 ('miike', 0.4935465157032013),
 ('flick', 0.492631196975708),
 ('picture', 0.48576119542121887),
 ('documentary', 0.4684869647026062),
 ('product', 0.468107134103775),
 ('nonetheless', 0.46794450283050537)]

In [ ]:
len(t.wv.vocab)


30

In [ ]:
def avg_word2vec(doc):
    return np.mean([w.wv[word] for word in doc if word in w.wv.vocab], axis=0)

In [ ]:
def AvgtagToVec(doc):
    return np.mean([t.wv[word] for word in doc if word in t.wv.vocab], axis = 0)

In [ ]:
from tqdm import tqdm

In [ ]:
W_vectorised = []
for i in tqdm(range(len(words))):
    W_vectorised.append(avg_word2vec(words[i]))

100%|██████████| 50000/50000 [00:12<00:00, 4072.23it/s]


In [ ]:
T_vectorised = []
for doc in tqdm(range(len(tag))):
    T_vectorised.append(AvgtagToVec(tag[i]))

100%|██████████| 50000/50000 [00:05<00:00, 9542.25it/s]


In [ ]:
len(T_vectorised)

50000

In [ ]:
wx=np.array(W_vectorised)
tx=np.array(T_vectorised)

In [ ]:
x=np.concatenate((wx,tx),axis=1)
x.shape

(50000, 400)

In [ ]:
y=data['sentiment']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .5, shuffle = False)

In [ ]:
x_train.shape

(25000, 400)

In [ ]:
clf = LogisticRegression()

In [ ]:
clf.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
%%time
kf = KFold(n_splits=10, random_state = 42, shuffle = True)
scores = cross_val_score(clf, x_train, y_train, cv = kf)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

CPU times: user 24.7 s, sys: 4.5 s, total: 29.2 s
Wall time: 15.2 s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
print('Cross-validation scores:', scores)
print('Cross-validation accuracy: {:.4f} (+/- {:.4f})'.format(scores.mean(), scores.std() * 2))

Cross-validation scores: [0.8584 0.858  0.8588 0.8648 0.8632 0.8636 0.8616 0.8516 0.8648 0.8504]
Cross-validation accuracy: 0.8595 (+/- 0.0098)


In [ ]:
predictions = clf.predict(x_test)

validation = dict()

validation['accuracy'] = accuracy_score(y_test, predictions)
validation['precision'] = precision_score(y_test, predictions, average='macro')
validation['recall'] = recall_score(y_test, predictions, average='macro')
validation['f1'] = f1_score(y_test, predictions, average='macro')

In [ ]:
print('Validation results:\n', '-' * 12)
for v in validation:
    print('{}: {:.5f}'.format(v.title(), validation[v]))

Validation results:
 ------------
Accuracy: 0.85840
Precision: 0.85851
Recall: 0.85838
F1: 0.85838
